In [0]:
!pip install newsapi-python

In [0]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
from datetime import date
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key='') #add your api key here

In [0]:
def get_sources(country):
    sources = newsapi.get_sources(country=country)
    sources = [x['id'] for x in sources['sources']]
    return sources

In [0]:
sources = get_sources('us')
print(sources)

['abc-news', 'al-jazeera-english', 'ars-technica', 'associated-press', 'axios', 'bleacher-report', 'bloomberg', 'breitbart-news', 'business-insider', 'buzzfeed', 'cbs-news', 'cnbc', 'cnn', 'cnn-es', 'crypto-coins-news', 'engadget', 'entertainment-weekly', 'espn', 'espn-cric-info', 'fortune', 'fox-news', 'fox-sports', 'google-news', 'hacker-news', 'ign', 'mashable', 'medical-news-today', 'msnbc', 'mtv-news', 'national-geographic', 'national-review', 'nbc-news', 'new-scientist', 'newsweek', 'new-york-magazine', 'next-big-future', 'nfl-news', 'nhl-news', 'politico', 'polygon', 'recode', 'reddit-r-all', 'reuters', 'techcrunch', 'techradar', 'the-american-conservative', 'the-hill', 'the-huffington-post', 'the-next-web', 'the-verge', 'the-wall-street-journal', 'the-washington-post', 'the-washington-times', 'time', 'usa-today', 'vice-news', 'wired']


In [0]:
def create_dataframe_last_30d(query, sources):
    fulldata = pd.DataFrame()
    for s in sources:
        #print(s)
        json_data = newsapi.get_everything(q=query,
                                          language='en',
                                            from_param=str(date.today() -timedelta(days=29)),
                                          to= str(date.today()),
                                            sources = s,
                                          page_size=100,
                                          page = 1,sort_by='relevancy'
                                          )
        if len(json_data['articles'])>0:
            data = pd.DataFrame(json_data['articles'])
            fulldata=pd.concat([fulldata,data])
    if len(fulldata)>0:
        fulldata['source'] = fulldata['source'].apply(lambda x : x['name'])
        fulldata['publishedAt'] = pd.to_datetime(fulldata['publishedAt'])
        fulldata = fulldata.drop_duplicates(subset='url').sort_values(by='publishedAt',ascending=False).reset_index()
    return fulldata

In [0]:
news_articles = create_dataframe_last_30d('Coronavirus', sources)
news_articles.shape

(4532, 9)

In [0]:
news_articles.head()

,index,source,author,title,description,url,urlToImage,publishedAt,content
0,30,CNBC,Holly Ellyatt,European markets set to open lower after US wa...,European markets are set to open lower Wednesd...,https://www.cnbc.com/2020/04/01/european-marke...,https://image.cnbcfm.com/api/v1/image/10646841...,2020-04-01 04:46:06+00:00,European markets are set to open lower Wednesd...
1,35,New York Magazine,Chas Danner,Coronavirus in New York: Latest Updates,The USNS Comfort hospital ship is now docked i...,https://nymag.com/intelligencer/2020/04/new-yo...,https://pixel.nymag.com/imgs/daily/intelligenc...,2020-04-01 04:45:00+00:00,"As of Tuesday night, there have been at least ..."
2,37,Google News,"info@hypebeast.com (HYPEBEAST), HYPEBEAST",Humble Bundle Offers Over $1000 USD Worth of G...,Humble Bundle is joining the fight against the...,http://feedproxy.google.com/~r/hypebeast/feed/...,None,2020-04-01 03:54:46+00:00,None
3,62,USA Today,"USA TODAY, Charles Trepany, USA TODAY",Coronavirus: Fashion companies from Dior to No...,Several of the world's biggest fashion brands ...,https://www.usatoday.com/story/entertainment/2...,https://www.gannett-cdn.com/presto/2020/03/27/...,2020-04-01 03:25:18+00:00,"Robert De Niro, Ben Stiller and La La Anthony ..."
4,78,ESPN Cric Info,Daniel Brettig,Newlands fallout has improved behaviour in Aus...,Australia's Test captain says the incidents in...,https://www.espncricinfo.com/story/_/id/289793...,https://a4.espncdn.com/combiner/i?img=%2Fi%2Fc...,2020-04-01 03:20:55+00:00,Australia Test captain Tim Paine believes the ...


In [0]:
news_articles.to_csv('/content/drive/My Drive/Colab Notebooks/Covid 19 Articles/new_articles.csv', 
                     index=False, header=True)